# install split_folders for Data splite in train, test and validation

In [78]:
!pip install split_folders

In [79]:
import splitfolders

In [80]:
input_folder ="flowers_dataset/Input_dataset"
output = "flowers_dataset/processed_data"
splitfolders.ratio(input_folder, output, seed=42, ratio=(.6, .2, .2))

Copying files: 4326 files [00:13, 317.97 files/s]


# import all necessory labaries

In [81]:
import tensorflow as tf

In [82]:
from tensorflow.keras.layers import Conv2D,Flatten,Dense,BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

# Define the height and width of the input images

In [83]:
img_height, img_width = (224,224)
batch_size = 32

train_data_dir = r'flowers_dataset\processed_data\train'
valid_data_dir = r'flowers_dataset\processed_data\val'
test_data_dir  = r'flowers_dataset\processed_data\test'

# Normalize the images based on the architecture

In [84]:
train_datagen =ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.4)
                                 
                        
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
     batch_size=batch_size,
    class_mode="categorical",
     subset='training')
                                 
valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation')

Found 1557 images belonging to 5 classes.
Found 342 images belonging to 5 classes.


In [85]:
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode="categorical",
    subset="validation")

Found 347 images belonging to 5 classes.


In [86]:
x,y = test_generator.next()
x.shape

(1, 224, 224, 3)

In [87]:
from tensorflow.keras import models, layers, optimizers, utils

# Defining the Model by giving a base model

In [88]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(train_generator,
                        epochs = 10)

Epoch 1/10
49/49 [==============================] - 160s 3s/step - loss: 1.3635 - accuracy: 0.6493
Epoch 2/10
49/49 [==============================] - 162s 3s/step - loss: 0.2768 - accuracy: 0.9009
Epoch 3/10
49/49 [==============================] - 145s 3s/step - loss: 0.1320 - accuracy: 0.9564
Epoch 4/10
49/49 [==============================] - 157s 3s/step - loss: 0.1513 - accuracy: 0.9517
Epoch 5/10
49/49 [==============================] - 151s 3s/step - loss: 0.0831 - accuracy: 0.9712
Epoch 6/10
49/49 [==============================] - 147s 3s/step - loss: 0.0837 - accuracy: 0.9704
Epoch 7/10
49/49 [==============================] - 160s 3s/step - loss: 0.0970 - accuracy: 0.9700
Epoch 8/10
49/49 [==============================] - 156s 3s/step - loss: 0.0607 - accuracy: 0.9769
Epoch 9/10
49/49 [==============================] - 151s 3s/step - loss: 0.0438 - accuracy: 0.9823
Epoch 10/10
49/49 [==============================] - 161s 3s/step - loss: 0.0452 - accuracy: 0.9840


# Compile the model

In [89]:
model.save('flowers_dataset\saved_Model\ResNet50_flowers.h5')

# Test_loss and Test_accuracy

In [90]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('\nTest accuracy:, test_acc')

347/347 - 55s - loss: 0.4950 - accuracy: 0.8934

Test accuracy:, test_acc
